In [1]:
# basic
import pandas as pd
import numpy as np
import time
import glob
from datetime import datetime 


# darts
from darts import TimeSeries
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler
import matplotlib.pyplot as plt


In [2]:
csv_files = glob.glob('/Users/dicod/semester_projects_02/Data/*.csv')
df = pd.DataFrame()

for file in csv_files:
    temp_df = pd.read_csv(file)
    df = pd.concat([df, temp_df], ignore_index=True)

In [3]:
df = df[['MTU (CET/CEST)','Day-ahead Price [EUR/MWh]']]
df = df.rename(columns={'MTU (CET/CEST)': 'date', 'Day-ahead Price [EUR/MWh]': 'Price'})
df['date'] = df['date'].apply(lambda x: x[0:16])

In [4]:
#df['Price'].fillna(df['Price'].mean(), inplace=True)
# why we use interpolate operation: Linear Interpolation simply means to estimate a missing value by connecting dots in a straight line in increasing order.
# In short, It estimates the unknown value in the same increasing order from previous values. The default method used by Interpolation is Linear.
df['Price'].fillna(df['Price'].interpolate(), inplace=True)

In [5]:
df.isnull().sum()

date     0
Price    0
dtype: int64

In [6]:
df_no_duplicates = df.drop_duplicates(subset='date')
df_no_duplicates.isnull().sum()

date     0
Price    0
dtype: int64

In [8]:
df_finish = df_no_duplicates
df_finish.head()

,date,Price
0,01.01.2020 00:00,33.42
1,01.01.2020 01:00,31.77
2,01.01.2020 02:00,31.57
3,01.01.2020 03:00,31.28
4,01.01.2020 04:00,30.85


In [9]:
df_finish.to_csv('clean_df.csv')

In [10]:
time_series = TimeSeries.from_dataframe(df_finish, time_col='date', value_cols = 'Price', fill_missing_dates=True, freq=None)

In [12]:
ts = time_series